### Leveraging LLMs to augment your data
#### Session: We will use the [Amazon UK Products Dataset](https://www.kaggle.com/datasets/asaniczka/amazon-uk-products-dataset-2023?resource=download) as an example, and extract entities from the products’ title
To do this, we will use Snowpark & Cortex

    -  Load product data from file to Snowflake 
    -  Filter out: Irrelevant categories, Products rated below 3.8 and Non-bestsellers
    -  Leverage Snowflake Cortex COMPLETE function and a Cortex provided LLM

***Note: All feature transformations using Snowpark are distributed operations.***

### Import Libraries

In [1]:
# Import the necessary libraries
# Data Science Libraries
#test git
import pandas as pd
import numpy as np

# Misc
import sys
from os import environ
import json

#from src.util.local import get_env_var_config
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.dataframe import col
from snowflake.snowpark.functions import  lit, concat, not_, count
from snowflake.snowpark.types import  StringType

#Cortex
from snowflake.cortex import Complete, ExtractAnswer, Sentiment, Summarize, Translate



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Establish Secure Connection to Snowflake

*Other connection options include Username/Password, MFA, OAuth, Okta, SSO. For more information, refer to the [Python Connector](https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-example) documentation.*

In [2]:
# Edit the connection.json before creating the session object below
# Create Snowflake Session object
#test
with open("connection.json") as f:
    connection_parameters = json.load(f)
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))


Connection Established with the following parameters:
User                        : SVADAKATH
Role                        : "ACCOUNTADMIN"
Database                    : "TESTING_STUFF"
Schema                      : "SNOWFLAKE_LLM"
Warehouse                   : "MY_HOL_WH"
Snowflake version           : 8.20.10
Snowpark for Python version : 1.14.0


In [5]:
#Data source: https://www.kaggle.com/datasets/asaniczka/amazon-uk-products-dataset-2023?resource=download

# # Load the CSV file into a DataFrame
df = pd.read_csv('../data/amz_uk_processed_data.csv', encoding='utf-8')

# Remove enclosing quotes from the column names
df.columns = df.columns.str.replace('"', '')
# df = df.sample(n=150000, random_state=42)
# df.to_csv('../data/amz_uk_processed_data.csv', encoding='utf-8', index=False)


product_df = df.reset_index()
product_df.count()
#testing

session.write_pandas(product_df, "AWS_UK_PRODUCTS", auto_create_table=True,overwrite=True)


In [6]:
# Set the display option to expand the maximum column width
pd.set_option('display.max_colwidth', 500)

# Specify the table name where we stored the Amazon UK product dataset
DEMO_TABLE = 'AWS_UK_PRODUCTS'
input_tbl = f"{session.get_current_database()}.{session.get_current_schema()}.{DEMO_TABLE}"
snowpark_df = session.table(input_tbl)

# Display the Product DataFrame
snowpark_df.select(count(col("ASIN"))).show()



---------------------
|"COUNT(""ASIN"")"  |
---------------------
|150000             |
---------------------



### Curate your data
-   Filter out irrelevant categories
-   Filter out products rated below 3.8
-   Filter out non-bestsellers

After doing this, we are left with ~4k products.

In [7]:


categories_to_delete = ['CD, Disc & Tape Players',
       'Light Bulbs', 'Bathroom Lighting',
       'Heating, Cooling & Air Quality',
       'Lab & Scientific Products',
       'Motorbike Batteries', 'Motorbike Boots & Luggage',
       'Motorbike Chassis',
       'Fireplaces, Stoves & Accessories', 'Blank Media Cases & Wallets', 'Car & Motorbike',
      'PA & Stage',
      'Wind Instruments',
        'Computer Printers', 'Ski Goggles',
       'Snowboards', 'Skiing Poles', 'Downhill Ski Boots',
       'Hiking Hand & Foot Warmers', 'Pet Supplies',
       'Plants, Seeds & Bulbs', 
       'Bird & Wildlife Care','Projectors', 'Graphics Cards', 'Computer Memory',
       'Motherboards', 'Power Supplies', 'CPUs', 'Computer Screws',
       'Streaming Clients', 'Barebone PCs',
       'SIM Cards',
       'Abrasive & Finishing Products',
       'Professional Medical Supplies', 'Cutting Tools',
       'Material Handling Products', 'Packaging & Shipping Supplies',
       'Power & Hand Tools', 'Agricultural Equipment & Supplies',
       'Tennis Shoes', 'Boating Footwear', 'Cycling Shoes', 'Water Coolers, Filters & Cartridges',
        'Flashes',
       'Computers, Components & Accessories', 'Motorbike Engines & Engine Parts',
       'Motorbike Drive & Gears', 'Motorbike Brakes',
       'Motorbike Exhaust & Exhaust Systems',
       'Motorbike Handlebars, Controls & Grips',
       'Mowers & Outdoor Power Tools', 'Kitchen & Bath Fixtures',
       'Rough Plumbing', 'Monitor Accessories', 'Cables & Accessories',
       'School & Educational Supplies',
       'Outdoor Heaters & Fire Pits', 'Window Treatments',
        'Mattress Pads & Toppers',
       "Children's Bedding", 'I/O Port Cards',
       'Computer Cases', 'KVM Switches', 'Printers & Accessories',
       'Telephones, VoIP & Accessories',
       'Industrial Electrical', 'Test & Measurement',
        'Electrical Power Accessories',
       'Radio Communication', 'Outdoor Rope Lights',
       'Vacuums & Floorcare', 'Large Appliances', 'Motorbike Lighting',
       'Motorbike Seat Covers', 'Motorbike Instruments',
       'Motorbike Electrical & Batteries', 'Lights and switches', 'Plugs',
       'Painting Supplies, Tools & Wall Treatments', 'Building Supplies',
       'Safety & Security', 'Tablet Accessories',
        'Decking & Fencing',
       'Thermometers & Meteorological Instruments',
       'Pools, Hot Tubs & Supplies',
       'Signs & Plaques',
       'Inflatable Beds, Pillows & Accessories', 'External Sound Cards',
       'Internal TV Tuner & Video Capture Cards',
       'External TV Tuners & Video Capture Cards',
       'Scanners & Accessories',
       'Professional Education Supplies',
       'Hydraulics, Pneumatics & Plumbing', 'Grocery',
       'Household Batteries, Chargers & Accessories',
        'Torches',
       'Sports Supplements', 'Ironing & Steamers',
         'Electrical',
       'Construction Machinery', 'Handmade Baby Products', 'USB Hubs',
        'Adapters',
       'Computer & Server Racks', 'Hard Drive Accessories',
       'Printer Accessories', 'Computer Memory Card Accessories',
       'Uninterruptible Power Supply Units & Accessories',
       'Recording & Computer',  'Office Paper Products', 'Ski Helmets',
       'Snowboard Boots', 'Snowboard Bindings', 'Downhill Skis',
       'Snow Sledding Equipment', 'Networking Devices',
        'Rugs, Pads & Protectors',
       'Slipcovers', 'External Optical Drives',
       'Internal Optical Drives', 'Network Cards', 'Data Storage',
       'Mobile Phones & Smartphones', 'Media Streaming Devices',
       'Hi-Fi Receivers & Separates', 'GPS, Finders & Accessories']

filtered_df = snowpark_df.filter(~col("CATEGORYNAME").isin(categories_to_delete))
filtered_df = filtered_df.filter(col("STARS")>= 3.8)
snowpark_final_df = filtered_df.filter(col("ISBESTSELLER")== '1')
snowpark_final_df.select(count(col("ASIN")).alias("result")).show()


------------
|"RESULT"  |
------------
|262       |
------------



### Extracting entities

We can then extract ***entities*** from the title of these products.
> Entity: a thing with distinct and independent existence

These entities could be anything — in the case of products they could be a color, a size, or a characteristic.

Two ways: [`Session SQL`](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.Session.sql) and [`Cortex Python Complete method`](https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/api/model/snowflake.cortex.Complete)


In [8]:
def complete_task(model_name,myprompttask,run_mode,snowpark_products_df):
    if run_mode == "SESSIONSQL_COMPLETE":

        #final_prompt = update_prompt (myprompttask)
        # cmd = f"""
        #     select ASIN,SNOWFLAKE.CORTEX.COMPLETE(?,?) as response FROM AWS_UK_PRODUCTS LIMIT 2
        #     """
        cmd = f"""
            select ASIN,TITLE, SNOWFLAKE.CORTEX.COMPLETE(?,CONCAT(?,': <review>', TITLE, '</review>')) as response FROM AWS_UK_PRODUCTS LIMIT 3
            """
    
        df_response = session.sql(cmd, params=[model_name, myprompttask]).collect()
    else:
        snowpark_products_df=snowpark_products_df.withColumn("concat_prompt",concat(lit(myprompttask).cast(StringType(3175)), \
                lit(': <review> '),snowpark_products_df.TITLE,lit(' </review>')))
        # snowpark_products_df1 = snowpark_products_df.filter((col("ASIN") == "B01HTH3C8S") | (col("ASIN") == "B09B97BPSW"))
        df_response =snowpark_products_df.withColumn("RESPONSE",Complete(lit(model_name),col("CONCAT_PROMPT"))).collect()
    return df_response

### Prompt
✨Create Prompt specifying ✨

1. Entities we need, and what they mean

     `description:` item detailed description, for example ‘high waist pants’, ‘outdoor plant pot’, ‘chef kitchen knife’.

     `exposure` is the time that a policy is exposed to risk, expressed in years.

     `type:` Item type, for example ‘women clothing’, ‘plant pot’, ‘kitchen knife’;

     `characteristic:` if present, item characteristics, for example ‘waterproof’, ‘adhesive’, ‘easy to use’;

     `measurement:` if present, dimensions of the item;

     `brand:` if present, brand of the item;

     `color:` if present, specific color of the item;

     `color_group:` if the color is present, this is the broader color group. For example, ‘navy blue’ is part of the color group ‘blue’, ‘burgundy’ is part of ‘red’, or ‘lilac’ is part of purple;

     `age_group:` target age group for the product, one of ‘babies’, ‘children’, ‘teenagers’, ‘adults’. If it is not clear whether the product is aimed at a specific age group, it should be for ‘adults’;

     `gender_group:` target gender for the product, one of ‘women’, ‘men’, ‘all’. If it is not clear whether the product is aimed at a specific
     gender, it should be for ‘all’
     


2. JSON output examples
        


In [9]:
#MAIN CODE:

#Cortex Models:  
    #'mixtral-8x7b'
    #'snowflake-arctic'
    #'mistral-large'
    #'llama3-8b'
    #'llama3-70b'
    #'reka-flash'
    #'mistral-7b'
    #'llama2-70b-chat'
    #'gemma-7b'
model = "mistral-large"

my_prompt = """You are an agent specialized in finding entities in online product descriptions.  
The user will give you a product description.  
Your task is to identify entities from the product description.  

The entities can be the following 9 types:  
1.description: item detailed description, for example ‘high waist pants’, ‘outdoor plant pot’, ‘chef kitchen knife’;
2.type: Item type, for example ‘women clothing’, ‘plant pot’, ‘kitchen knife’;
3.characteristic: if present, item characteristics, for example ‘waterproof’, ‘adhesive’, ‘easy to use’;
4.measurement: if present, dimensions of the item;
5.brand: if present, brand of the item;
6.color: if present, specific color of the item;
7.color_group: if the color is present, this is the broader color group. For example, ‘navy blue’ is part of the color group ‘blue’, ‘burgundy’ is part of ‘red’, or ‘lilac’ is part of purple;
8.age_group: target age group for the product, one of ‘babies’, ‘children’, ‘teenagers’, ‘adults’. If it is not clear whether the product is aimed at a specific age group, it should be for ‘adults’;
9.gender_group: target gender for the product, one of ‘women’, ‘men’, ‘all’. If it is not clear whether the product is aimed at a specific gender, it should be for ‘all’

You must return a JSON output containing for every type of entity found a list of values.

If you cannot find an entity type, return an empty array for this entity.  
If you found one entity of this type, return an array with one value.  
If you found 2 entities of this type, return an array with 2 values.  
Etc.  

Only use lower cases letters when defining entities values, and remove adjectives and specificities from values to try and have the simplest words or groups of words.

Examples:
    1. Description: "YUVORA 3D Brick Wall Stickers | PE Foam Fancy Wallpaper for Walls,
         Waterproof & Self Adhesive, White Color 3D Latest Unique Design Wallpaper for Home (70*70 CMT)"
        Expected result:
        {
            "description": ["3d brick wall sticker"],
            "type": ["wall sticker", "wallpaper"],
            "brand": ["yuvora"],
            "characteristic": ["waterproof", "self-adhesive", "fancy"],
            "color": ["white"],
            "color_group": ["white"],
            "age_group": ["adults"],
            "gender_group": ["all"]
        }
    2. Description: "Marks & Spencer Girls Pyjama Sets T86_2561C_Navy Mix_9-10Y"
        Expected result:
        {
            "description": ["pyjama sets"],
            "type": ["pyjamas"],
            "brand": ["marks & spencer"],
            "characteristic": [],
            "color": ["navy"],
            "color_group": ["blue"],
            "age_group": ["children"],
            "gender_group": ["women"]
        }
    3. Description: "Star Trek 50th Anniversary Cereamic Storage Jar"
        Expected result:
        {
            "description": ["star trek storage jar"],
            "type": ["storage jar"],
            "brand": [],
            "characteristic": ["ceramic", "star trek"],
            "color": [],
            "color_group": ],
            "age_group": ["adults"],
            "gender_group": ["all"]
        }'
"""
#run_mode could be "SESSIONSQL_COMPLETE" or "SNOWPARK_COMPLETE_METHOD"
from snowflake.snowpark.functions import random

run_mode="SNOWPARK_COMPLETE_METHOD"
column_name="TITLE"
llm_response = complete_task(model,my_prompt,run_mode,snowpark_final_df.order_by(random()).limit(5))
llm_response_df=pd.DataFrame(llm_response)
llm_response_df


Complete() is experimental since 1.0.12. Do not use it in production. 


,index,ASIN,TITLE,IMGURL,PRODUCTURL,STARS,REVIEWS,PRICE,ISBESTSELLER,BOUGHTINLASTMONTH,CATEGORYNAME,CONCAT_PROMPT,RESPONSE
0,238010,B07D4MJHZ3,"Bemece Universal Rain Cover for Pushchair Stroller Buggy Pram, Baby Travel Weather Shield",https://m.media-amazon.com/images/I/61+6FKQfZsL._AC_UL320_.jpg,https://www.amazon.co.uk/dp/B07D4MJHZ3,4.2,6012,15.80,True,1000,Baby,"You are an agent specialized in finding entities in online product descriptions. \nThe user will give you a product description. \nYour task is to identify entities from the product description. \n\nThe entities can be the following 9 types: \n1.description: item detailed description, for example ‘high waist pants’, ‘outdoor plant pot’, ‘chef kitchen knife’;\n2.type: Item type, for example ‘women clothing’, ‘plant pot’, ‘kitchen knife’;\n3.characteristic: if present, item characteristics...","\n\n{\n ""description"": [""universal rain cover for pushchair stroller buggy pram"", ""baby travel weather shield""],\n ""type"": [""rain cover"", ""weather shield""],\n ""brand"": [""bemece""],\n ""characteristic"": [""universal"", ""travel""],\n ""color"": [],\n ""color_group"": [],\n ""age_group"": [""babies""],\n ""gender_group"": [""all""]\n}"
1,222844,B0B97DJ8X5,"Januts Take-Apart Construction Vehicles Mini Engineering Toy Play Set with 4 Pack, Excavator Road Roller Bulldozer Ground Drilling Rig DIY Building Educational Toys for Boys Girls (Blue)",https://m.media-amazon.com/images/I/71gS04ErQzL._AC_UL320_.jpg,https://www.amazon.co.uk/dp/B0B97DJ8X5,4.5,25,14.99,True,50,Kids' Play Vehicles,"You are an agent specialized in finding entities in online product descriptions. \nThe user will give you a product description. \nYour task is to identify entities from the product description. \n\nThe entities can be the following 9 types: \n1.description: item detailed description, for example ‘high waist pants’, ‘outdoor plant pot’, ‘chef kitchen knife’;\n2.type: Item type, for example ‘women clothing’, ‘plant pot’, ‘kitchen knife’;\n3.characteristic: if present, item characteristics...","\n\n{\n ""description"": [""take-apart construction vehicles"", ""mini engineering toy play set"", ""excavator road roller bulldozer ground drilling rig"", ""diy building educational toys""],\n ""type"": [""construction vehicles"", ""toy play set"", ""educational toys""],\n ""brand"": [""januts""],\n ""characteristic"": [""take-apart"", ""mini"", ""diy"", ""building"", ""educational""],\n ""measurement"": [""4 pack""],\n ""color"": [""blue""],\n ""color_group"": [""blue""],\n ""age_group"": [""children""],\n ""gend..."
2,252767,B07BJMNGTP,Ram® Large 2L Hot Water Bottle with Soft Fleece Cover,https://m.media-amazon.com/images/I/61guxR7uc9L._AC_UL320_.jpg,https://www.amazon.co.uk/dp/B07BJMNGTP,4.4,10531,5.49,True,3000,Health & Personal Care,"You are an agent specialized in finding entities in online product descriptions. \nThe user will give you a product description. \nYour task is to identify entities from the product description. \n\nThe entities can be the following 9 types: \n1.description: item detailed description, for example ‘high waist pants’, ‘outdoor plant pot’, ‘chef kitchen knife’;\n2.type: Item type, for example ‘women clothing’, ‘plant pot’, ‘kitchen knife’;\n3.characteristic: if present, item characteristics...","\n\n{\n ""description"": [""large hot water bottle"", ""fleece cover""],\n ""type"": [""hot water bottle""],\n ""brand"": [""ram""],\n ""characteristic"": [""soft""],\n ""measurement"": [""2l""],\n ""color"": [],\n ""color_group"": [],\n ""age_group"": [""adults""],\n ""gender_group"": [""all""]\n}"
3,297305,B07SX2PRHL,"Hydration Bladder, 3 L 2 L 1.5 L Water Bladder BPA Free, 3 Liter Large Opening Water Reservoir, Leak Proof Military Water Storage Bladder Bag for Cycling Hiking Camping Biking Running",https://m.media-amazon.com/images/I/71n59dnoVCL._AC_UL320_.jpg,https://www.amazon.co.uk/dp/B07SX2PRHL,4.3,3711,11.89,True,900,Luggage and travel gear,"You are an agent speciali

In [10]:
# Convert the JSON string column to a JSON object
llm_response_df['json_col'] = llm_response_df['RESPONSE'].apply(json.loads)

# Drop the original JSON string column if no longer needed
llm_response_df.drop(columns=['RESPONSE'], inplace=True)

# Display the modified DataFrame
print(llm_response_df)

    index        ASIN  \
0  238010  B07D4MJHZ3   
1  222844  B0B97DJ8X5   
2  252767  B07BJMNGTP   
3  297305  B07SX2PRHL   
4  103953  B00DGPDCWE   

                                                                                                                                                                                        TITLE  \
0                                                                                                   Bemece Universal Rain Cover for Pushchair Stroller Buggy Pram, Baby Travel Weather Shield   
1  Januts Take-Apart Construction Vehicles Mini Engineering Toy Play Set with 4 Pack, Excavator Road Roller Bulldozer Ground Drilling Rig DIY Building Educational Toys for Boys Girls (Blue)   
2                                                                                                                                       Ram® Large 2L Hot Water Bottle with Soft Fleece Cover   
3     Hydration Bladder, 3 L 2 L 1.5 L Water Bladder BPA Free, 3 Liter Large 